In [53]:
import pandas as pd
import numpy as np
import time
from datetime import datetime as dt
from sklearn.neural_network import MLPRegressor
pd.set_option('display.max_rows', 500)
import matplotlib.pyplot as plt
from sklearn.preprocessing import MinMaxScaler, RobustScaler
from sklearn.metrics import mean_squared_error
from sklearn.ensemble import RandomForestRegressor
from sklearn.linear_model import LinearRegression
from sklearn.linear_model import Ridge
from sklearn import linear_model
from sklearn.model_selection import train_test_split
import statsmodels.api as sm
from sklearn.metrics import r2_score
from sklearn.metrics import mean_squared_error
from sklearn.datasets import make_regression
import statsmodels.api as sm
from sklearn.neural_network import MLPClassifier
from sklearn.metrics import accuracy_score
from sklearn.svm import SVC


pd.set_option("display.max_columns", None)

In [12]:
# Reading the csv file, and modifying the index

data = pd.read_csv("telco.csv")
data = data.set_index("customerID")

data

,gender,SeniorCitizen,Partner,Dependents,tenure,PhoneService,MultipleLines,InternetService,OnlineSecurity,OnlineBackup,DeviceProtection,TechSupport,StreamingTV,StreamingMovies,Contract,PaperlessBilling,PaymentMethod,MonthlyCharges,TotalCharges,Churn
customerID,,,,,,,,,,,,,,,,,,,,
7590-VHVEG,Female,0,Yes,No,1,No,No phone service,DSL,No,Yes,No,No,No,No,Month-to-month,Yes,Electronic check,29.85,29.85,No
5575-GNVDE,Male,0,No,No,34,Yes,No,DSL,Yes,No,Yes,No,No,No,One year,No,Mailed check,56.95,1889.5,No
3668-QPYBK,Male,0,No,No,2,Yes,No,DSL,Yes,Yes,No,No,No,No,Month-to-month,Yes,Mailed check,53.85,108.15,Yes
7795-CFOCW,Male,0,No,No,45,No,No phone service,DSL,Yes,No,Yes,Yes,No,No,One year,No,Bank transfer (automatic),42.30,1840.75,No
9237-HQITU,Female,0,No,No,2,Yes,No,Fiber optic,No,No,No,No,No,No,Month-to-month,Yes,Electronic check,70.70,151.65,Yes
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6840-RESVB,Male,0,Yes,Yes,24,Yes,Yes,DSL,Yes,No,Yes,Yes,Yes,Yes,One year,Yes,Mailed check,84.80,1990.5,No
2234-XADUH,Female,0,Yes,Yes,72,Yes,Yes,Fiber optic,No,Yes,Yes,No,Yes,Yes,One year,Yes,Credit card (automatic),103.20,7362.9,No
4801-JZAZL,Female,0,Yes,Yes,11,No,No phone service,DSL,Yes,No,No,No,No,No,Month-to-month,Yes,Electronic check,29.60,346.45,No


In [13]:
# Converting Yes/No features to 1/0

data.Partner.replace(("Yes", "No"), (1, 0), inplace=True)
data.Dependents.replace(("Yes", "No"), (1, 0), inplace=True)
data.PhoneService.replace(("Yes", "No"), (1, 0), inplace=True)
data.PaperlessBilling.replace(("Yes", "No"), (1, 0), inplace=True)
data.Churn.replace(("Yes", "No"), (1, 0), inplace=True)

# Removing the 11 entries with missing TotalCharges data
data = data[data.TotalCharges!=" "]

# Converting TotalCharges column into float type
data = data.astype({"TotalCharges":"float"})

# Creating dummy variables for the other features, since those columns have more than 2 distinct values
data = pd.get_dummies(data, drop_first=False)

# Dropping some dummy columns to prevent multicollinearity
data = data.drop(columns=["gender_Female", "MultipleLines_No phone service", "OnlineSecurity_No internet service", "OnlineBackup_No internet service", "DeviceProtection_No internet service", "TechSupport_No internet service", "StreamingTV_No internet service", "StreamingMovies_No internet service"])

# Put the target variable at the end of the table
cols = data.columns.tolist()
cols = cols[:8] + cols[9:] + [cols[8]]
data = data[cols]

data

,SeniorCitizen,Partner,Dependents,tenure,PhoneService,PaperlessBilling,MonthlyCharges,TotalCharges,Churn,gender_Male,MultipleLines_No,MultipleLines_Yes,InternetService_DSL,InternetService_Fiber optic,InternetService_No,OnlineSecurity_No,OnlineSecurity_Yes,OnlineBackup_No,OnlineBackup_Yes,DeviceProtection_No,DeviceProtection_Yes,TechSupport_No,TechSupport_Yes,StreamingTV_No,StreamingTV_Yes,StreamingMovies_No,StreamingMovies_Yes,Contract_Month-to-month,Contract_One year,Contract_Two year,PaymentMethod_Bank transfer (automatic),PaymentMethod_Credit card (automatic),PaymentMethod_Electronic check,PaymentMethod_Mailed check
customerID,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
7590-VHVEG,0,1,0,1,0,1,29.85,29.85,0,0,0,0,1,0,0,1,0,0,1,1,0,1,0,1,0,1,0,1,0,0,0,0,1,0
5575-GNVDE,0,0,0,34,1,0,56.95,1889.50,0,1,1,0,1,0,0,0,1,1,0,0,1,1,0,1,0,1,0,0,1,0,0,0,0,1
3668-QPYBK,0,0,0,2,1,1,53.85,108.15,1,1,1,0,1,0,0,0,1,0,1,1,0,1,0,1,0,1,0,1,0,0,0,0,0,1
7795-CFOCW,0,0,0,45,0,0,42.30,1840.75,0,1,0,0,1,0,0,0,1,1,0,0,1,0,1,1,0,1,0,0,1,0,1,0,0,0
9237-HQITU,0,0,0,2,1,1,70.70,151.65,1,0,1,0,0,1,0,1,0,1,0,1,0,1,0,1,0,1,0,1,0,0,0,0,1,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6840-RESVB,0,1,1,24,1,1,84.80,1990.50,0,1,0,1,1,0,0,0,1,1,0,0,1,0,1,0,1,0,1,0,1,0,0,0,0,1
2234-XADUH,0,1,1,72,1,1,103.20,7362.90,0,0,0,1,0,1,0,1,0,0,1,0,1,1,0,0,1,0,1,0,1,0,0,1,0,0
4801-JZAZL,0,1,1,11,0,1,29.60,346.45,0,0,0,0,1,0,0,0,1,1,0,1,0,1,0,1,0,1,0,1,0,0,0,0,1,0


In [17]:
# Put the target variable at the end of the table
cols = data.columns.tolist()
cols = cols[:8] + cols[9:] + [cols[8]]
data = data[cols]

data

,SeniorCitizen,Partner,Dependents,tenure,PhoneService,PaperlessBilling,MonthlyCharges,TotalCharges,gender_Male,MultipleLines_No,MultipleLines_Yes,InternetService_DSL,InternetService_Fiber optic,InternetService_No,OnlineSecurity_No,OnlineSecurity_Yes,OnlineBackup_No,OnlineBackup_Yes,DeviceProtection_No,DeviceProtection_Yes,TechSupport_No,TechSupport_Yes,StreamingTV_No,StreamingTV_Yes,StreamingMovies_No,StreamingMovies_Yes,Contract_Month-to-month,Contract_One year,Contract_Two year,PaymentMethod_Bank transfer (automatic),PaymentMethod_Credit card (automatic),PaymentMethod_Electronic check,PaymentMethod_Mailed check,Churn
customerID,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
7590-VHVEG,0,1,0,1,0,1,29.85,29.85,0,0,0,1,0,0,1,0,0,1,1,0,1,0,1,0,1,0,1,0,0,0,0,1,0,0
5575-GNVDE,0,0,0,34,1,0,56.95,1889.50,1,1,0,1,0,0,0,1,1,0,0,1,1,0,1,0,1,0,0,1,0,0,0,0,1,0
3668-QPYBK,0,0,0,2,1,1,53.85,108.15,1,1,0,1,0,0,0,1,0,1,1,0,1,0,1,0,1,0,1,0,0,0,0,0,1,1
7795-CFOCW,0,0,0,45,0,0,42.30,1840.75,1,0,0,1,0,0,0,1,1,0,0,1,0,1,1,0,1,0,0,1,0,1,0,0,0,0
9237-HQITU,0,0,0,2,1,1,70.70,151.65,0,1,0,0,1,0,1,0,1,0,1,0,1,0,1,0,1,0,1,0,0,0,0,1,0,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6840-RESVB,0,1,1,24,1,1,84.80,1990.50,1,0,1,1,0,0,0,1,1,0,0,1,0,1,0,1,0,1,0,1,0,0,0,0,1,0
2234-XADUH,0,1,1,72,1,1,103.20,7362.90,0,0,1,0,1,0,1,0,0,1,0,1,1,0,0,1,0,1,0,1,0,0,1,0,0,0
4801-JZAZL,0,1,1,11,0,1,29.60,346.45,0,0,0,1,0,0,0,1,1,0,1,0,1,0,1,0,1,0,1,0,0,0,0,1,0,0


In [21]:
#Split the data set as train and test
X_train, X_test, y_train, y_test = train_test_split(data.values[:,:-1], data.values[:,-1], train_size = 0.85, test_size=0.15, random_state=91)
train = np.concatenate((X_train, y_train.reshape((-1,1))), axis=1)
test = np.concatenate((X_test, y_test.reshape((-1,1))), axis = 1)

[[0. 1. 0. ... 0. 1. 0.]
 [0. 0. 0. ... 0. 0. 1.]
 [0. 1. 1. ... 0. 0. 1.]
 ...
 [1. 0. 0. ... 0. 1. 0.]
 [0. 1. 1. ... 0. 0. 1.]
 [0. 1. 1. ... 0. 1. 0.]] [0. 0. 0. ... 1. 0. 0.]


In [35]:
#Scale the data
scaler = RobustScaler()
scaler.fit(train)
scaled_train = scaler.transform(train)
scaled_test = scaler.transform(test)

In [47]:
learning_rates = [10**(-3)]
results = {"accuracy_train":[] ,"accuracy_test":[], "run_time":[], "n_iterations":[], "learning_rate":[]}
for learning_rate in learning_rates:
    t1 = time.time()
    neural_net = MLPClassifier(shuffle = True,learning_rate_init = learning_rate,max_iter=2000, hidden_layer_sizes = (128,128,128,128,128), solver = "adam", tol = 9e-6, batch_size = 1024
                         ).fit(scaled_train[:,:-1], scaled_train[:,-1])
    t2 = time.time()
    run_time = t2-t1
    train_actual = scaled_train[:,-1]
    train_predict = neural_net.predict(scaled_train[:,:-1])
    test_actual = scaled_test[:,-1]
    test_predict = neural_net.predict(scaled_test[:,:-1])
    accuracy_train = accuracy_score(train_actual,train_predict)
    accuracy_test = accuracy_score(test_actual,test_predict)
    results["accuracy_train"].append(accuracy_train)
    results["accuracy_test"].append(accuracy_test)
    results["run_time"].append(run_time)
    results["n_iterations"].append(neural_net.n_iter_)
    results["learning_rate"].append(learning_rate)
    print("Neural Net trained.")

Neural Net trained.
Neural Net trained.


In [48]:
df = pd.DataFrame(results)
df

,accuracy_train,accuracy_test,run_time,n_iterations,learning_rate
0,0.945458,0.751659,12.019043,107,0.0010
1,0.959344,0.748815,20.012580,182,0.0005


SVM classifier


In [58]:
results = {"accuracy_train":[] ,"accuracy_test":[], "run_time":[], "C_val":[], "kernel_val":[]}
C_val = [0.01 ,0.1, 1, 5, 10]
kernel_val = ["linear","poly", "rbf", "sigmoid"]
counter = 0
for kernel in kernel_val:
    for c in C_val:
            t1 = time.time()
            model = SVC(kernel = kernel, C = c, tol = 0.0001).fit(scaled_train[:,:-1], scaled_train[:,-1])
            t2 = time.time()
            run_time = t2-t1
            
            train_actual = scaled_train[:,-1]
            train_predict = model.predict(scaled_train[:,:-1])
            test_actual = scaled_test[:,-1]
            test_predict = model.predict(scaled_test[:,:-1])
            
            acc_train = accuracy_score(train_actual,train_predict)
            acc_test = accuracy_score(test_actual,test_predict)


            results["accuracy_train"].append(acc_train)
            results["accuracy_test"].append(acc_test)

            results["run_time"].append(run_time)
            results["C_val"].append(c)
            results["kernel_val"].append(kernel)
            counter += 1
            print("Model-{} trained".format(counter))

Model-1 trained
Model-2 trained
Model-3 trained
Model-4 trained
Model-5 trained
Model-6 trained
Model-7 trained
Model-8 trained
Model-9 trained
Model-10 trained
Model-11 trained
Model-12 trained
Model-13 trained
Model-14 trained
Model-15 trained
Model-16 trained
Model-17 trained
Model-18 trained
Model-19 trained
Model-20 trained


In [59]:
df = pd.DataFrame(results)
df.sort_values("accuracy_test", inplace = True, ascending=False)
df

,accuracy_train,accuracy_test,run_time,C_val,kernel_val
2,0.797725,0.818957,0.885776,1.00,linear
3,0.798059,0.818009,1.500671,5.00,linear
4,0.798059,0.818009,2.049010,10.00,linear
1,0.796052,0.818009,0.625316,0.10,linear
0,0.796553,0.815166,0.633596,0.01,linear
7,0.816463,0.814218,0.836873,1.00,poly
11,0.796888,0.812322,0.888013,0.10,rbf
13,0.832692,0.811374,1.171867,5.00,rbf
12,0.810775,0.810427,0.952085,1.00,rbf
16,0.788355,0.808531,0.923571,0.10,sigmoid
